### Bias Detection and Fairness Evaluation on Heart Failure Prediction Dataset (Kaggle) using FairMLhealth
(Source: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction/data)

In [1]:
import pandas as pd

# Load X_test set
X_test = pd.read_csv("./data_splits/X_test.csv")
y_test = pd.read_csv("./data_splits/y_test.csv")

In [2]:
import fairmlhealth
import aif360
print("Environment setup successful")

Environment setup successful


In [3]:
#have a look at the details of fairmlhealth - especially the version
!pip show fairmlhealth

Name: fairmlhealth
Version: 1.0.2
Summary: Health-centered variation analysis
Home-page: https://github.com/KenSciResearch/fairMLHealth
Author: Christine Allen
Author-email: ca.magallen@gmail.com
License: 
Location: c:\users\patri\appdata\roaming\python\python310\site-packages
Requires: aif360, ipython, jupyter, numpy, pandas, requests, scikit-learn, scipy
Required-by: 


In [4]:
#have a look at the modules that are within fairmlhealth

print(dir(fairmlhealth))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [5]:
#load necessary modules 

#import module measure to use measure.summary for bias detection
from fairmlhealth import measure

#import module for investigation of individual cohorts 
from fairmlhealth.__utils import iterate_cohorts

#import FairRanges to flag high values
from fairmlhealth.__utils import FairRanges

# Wrap the fairness summary function for cohort-wise analysis
@iterate_cohorts
def cohort_summary(**kwargs):
    return measure.summary(**kwargs)

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration 

During the execution of FairMLHealth and AIF360, several runtime warnings were raised (e.g., “AdversarialDebiasing will be unavailable” due to the absence of TensorFlow, and deprecation warnings from the inFairness package regarding PyTorch’s functorch.vmap). These warnings do not affect the fairness metrics or results presented in this study, as the unavailable components were not used. To maintain clarity of output, the warnings were silenced programmatically, and the analysis was conducted without issue.

In [6]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", module="inFairness")
warnings.filterwarnings("ignore", message="AdversarialDebiasing will be unavailable")

### Traditional Machine Learning Models - KNN & DT

#### K-nearest neighbors - KNN

In [7]:
import pandas as pd

# Load KNN results
knn_df = pd.read_csv("HeartFailureData_50_50_PCA_KNN_predictions.csv")

print(knn_df.head())

   Sex  y_true    y_prob  y_pred
0    1       1  0.935632       1
1    1       1  0.718133       1
2    1       1  0.947130       1
3    1       1  0.379473       0
4    0       0  0.238673       0


In [8]:
# Extract common columns
y_true_knn = knn_df["y_true"].values
y_prob_knn = knn_df["y_prob"].values
y_pred_knn = knn_df["y_pred"].values
gender_knn = knn_df["Sex"].values

# Use gender_knn as the protected attribute (0/1 as in your CSV)
protected_attr_knn = gender_knn

In [9]:
knn_bias = measure.summary(
    X=X_test,
    y_true=y_true_knn,
    y_pred=y_pred_knn,
    y_prob=y_prob_knn,
    prtc_attr=protected_attr_knn,
    pred_type="classification",
    priv_grp=1,
    sig_fig=4,
    skip_if=True,   # skip inconsistency metrics that cause NearestNeighbors error
    skip_performance=True
)

print(knn_bias)

                                                        Value
Metric         Measure                                       
Group Fairness AUC Difference                          0.0350
               Balanced Accuracy Difference           -0.0750
               Balanced Accuracy Ratio                 0.9145
               Disparate Impact Ratio                  0.2650
               Equal Odds Difference                  -0.1875
               Equal Odds Ratio                        0.6250
               Positive Predictive Parity Difference  -0.2759
               Positive Predictive Parity Ratio        0.7073
               Statistical Parity Difference          -0.4380
Data Metrics   Prevalence of Privileged Class (%)     79.0000


In [10]:
# 2) Custom scenario oriented bounds

custom_ranges = {
    "tpr diff": (-0.03, 0.03),
    "fpr diff": (-0.03, 0.03),
    "equal odds difference": (-0.04, 0.04),
    "statistical parity difference": (-0.05, 0.05),
    "disparate impact ratio": (0.9, 1.1),
    "selection ratio": (0.9, 1.1),
    "auc difference": (-0.02, 0.02),
    "balanced accuracy difference": (-0.02, 0.02),
}

bounds = FairRanges().load_fair_ranges(custom_ranges=custom_ranges)

In [11]:
#for highlighting metrics outside of the thresholds
#  restore Styler.set_precision to adjust the highlighting color in the styled table
import pandas as pd, numpy as np

Styler = type(pd.DataFrame({"_":[0]}).style)  

if not hasattr(Styler, "set_precision"):
    def _set_precision(self, precision=4):
        try:
            return self.format(precision=precision)
        except TypeError:
            return self.format(formatter=lambda x:
                f"{x:.{precision}g}" if isinstance(x, (int, float, np.floating)) else x
            )
    setattr(Styler, "set_precision", _set_precision)

In [12]:
#Flag metrics outside acceptable fairness bounds in current table 

from fairmlhealth.__utils import Flagger

class MyFlagger(Flagger):
    def reset(self):
        super().reset()
        self.flag_color = "#491ee6"   
        self.flag_type = "background-color"

styled_knn = MyFlagger().apply_flag(
    df=knn_bias,
    caption="KNN Fairness (Gender)",
    boundaries=bounds,
    sig_fig=4,
    as_styler=True
)
styled_knn

## Fairness Analysis – KNN Model (by Gender)

This table summarizes **fairness metrics** for the KNN model, comparing performance between the unprivileged group (female) and the privileged group (male).  

---

### 1. AUC & Balanced Accuracy
- **AUC Difference (0.0350):** Small disparity in ranking quality across genders, but males slightly benefit from better ranking performance.  
- **Balanced Accuracy Difference (−0.0750), Ratio (0.9145):** Females experience noticeably **lower balanced accuracy**, showing reduced fairness in how the model handles positive and negative cases.  

---

### 2. Group Fairness Metrics
- **Disparate Impact Ratio (0.2650):** Well below the fairness threshold (0.80–1.25). This means the selection rate for females is **much lower**, signaling strong under-representation.  
- **Equal Odds Difference (−0.1875), Ratio (0.6250):** Significant imbalance in error rates (TPR and FPR) across genders, disadvantaging females.  
- **Statistical Parity Difference (−0.4380):** Negative value confirms females are **selected far less often** than males.  

---

### 3. Predictive Parity
- **Positive Predictive Parity Difference (−0.2759), Ratio (0.7073):** Females have **lower precision** than males, meaning predictions for females are less reliable.  

---

### **Interpretation**
- The KNN model shows **systematic gender bias**, particularly disadvantaging the **female (unprivileged) group**.  
- Females face:
  - Lower balanced accuracy.  
  - Much lower selection rates.  
  - Weaker precision and reliability of predictions.  
- Males (privileged group) benefit from consistently more favorable outcomes across fairness measures.  

---

### **Summary**
The fairness metrics reveal that KNN is **not gender-fair**. It disproportionately disadvantages females through **reduced selection, lower precision, and weaker balanced accuracy**, while males are treated more favorably. These disparities highlight a **serious fairness concern**, making bias mitigation necessary before practical use.  

---

In [13]:
print("FairMLHealth Stratified Bias Table - KNN")
measure.bias(X_test, y_test, y_pred_knn, features=['Sex'], flag_oor=False)

FairMLHealth Stratified Bias Table - KNN


,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,Sex,0,0.075,1.0935,0.0375,1.600,0.2759,1.4138,0.438,3.774,0.1875,1.2812
1,Sex,1,-0.075,0.9145,-0.0375,0.625,-0.2759,0.7073,-0.438,0.265,-0.1875,0.7805


## Stratified Bias Analysis – KNN by Gender

---

### 1. Balanced Accuracy
- **Females (0):** Balanced Accuracy Difference = **+0.0750**, Ratio = **1.0935**  
- **Males (1):** Balanced Accuracy Difference = **−0.0750**, Ratio = **0.9145**  
- ➝ The model achieves **higher balanced accuracy for females**, suggesting better handling of true positives and true negatives for the unprivileged group.

---

### 2. False Positive Rate (FPR)
- **Females (0):** FPR Difference = **+0.0375**, Ratio = **1.6000**  
- **Males (1):** FPR Difference = **−0.0375**, Ratio = **0.6250**  
- ➝ Females face a **higher false positive rate**, meaning they are more often incorrectly flagged as positive compared to males.

---

### 3. Positive Predictive Value (PPV / Precision)
- **Females (0):** PPV Difference = **+0.2759**, Ratio = **1.4138**  
- **Males (1):** PPV Difference = **−0.2759**, Ratio = **0.7073**  
- ➝ Predictions for females are **more reliable (higher precision)**, while for males, predictions are less trustworthy.

---

### 4. Selection Rate
- **Females (0):** Selection Difference = **+0.4380**, Ratio = **3.7740**  
- **Males (1):** Selection Difference = **−0.4380**, Ratio = **0.2650**  
- ➝ The model selects **females at a much higher rate** than males, indicating strong preferential treatment toward the unprivileged group.

---

### 5. True Positive Rate (TPR / Sensitivity)
- **Females (0):** TPR Difference = **+0.1875**, Ratio = **1.2812**  
- **Males (1):** TPR Difference = **−0.1875**, Ratio = **0.7805**  
- ➝ Females benefit from a **higher sensitivity**, meaning they are more likely to be correctly identified when positive, while males face more missed detections.

---

### **Summary**
- The KNN model demonstrates a **reverse bias** in this setup, **favoring females (unprivileged group)** over males (privileged group).  
- Females experience **higher balanced accuracy, higher precision, greater selection rates, and stronger sensitivity**.  
- However, this comes at the cost of a **higher false positive rate for females**, meaning they are more likely to be incorrectly flagged.  
- Males, on the other hand, face **under-selection and reduced predictive reliability**, highlighting a clear imbalance.

In short, the model’s fairness issue here is not traditional male privilege but rather a **systematic skew toward females**, creating inequity in the opposite direction.

---

In [14]:
from fairmlhealth import measure
import pandas as pd
from IPython.display import display  

# convert gender into DataFrame with a clear column name to get a nice table as output
gender_df = pd.DataFrame({"Sex": X_test["Sex"].astype(int)})


# Get the stratified table
perf_table_knn = measure.performance(
    X=gender_df,
    y_true=y_test,
    y_pred=y_pred_knn,
    y_prob=y_prob_knn
)

# Replace NaN with a dash
perf_table_knn = perf_table_knn.fillna("—")

# display pretty table
display(perf_table_knn)

C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\fairmlhealth\__utils.py:57: UserWarning: Possible error in column(s) ['Sex']. DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,184.0,0.5543,0.5054,0.8750,0.8821,0.0854,—,0.9247,0.9348,0.8431
1,Sex,0,38.0,0.1579,0.1579,0.8947,0.6667,0.0625,—,0.6667,0.9531,0.6667
2,Sex,1,146.0,0.6575,0.5959,0.8699,0.8962,0.1000,—,0.9425,0.9181,0.8542


## Stratified Performance Analysis – KNN by Gender

This table shows the **stratified performance metrics** of the KNN model across gender groups.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.

---

### 1. Overall Performance (All Features)
- **Accuracy (0.8750)** and **F1-score (0.8821)** indicate strong overall performance.  
- **ROC AUC (0.9348)** demonstrates excellent discriminatory ability.  
- **Precision (0.9247)** shows predictions are highly reliable.  
- **TPR (0.8431)** reflects solid sensitivity, though subgroup analysis reveals disparities.

---

### 2. Subgroup Comparison

| Metric        | Female (0) | Male (1) | Interpretation |
|---------------|------------|----------|----------------|
| **Accuracy**  | 0.8947     | 0.8699   | Accuracy is slightly higher for females. |
| **F1-Score**  | 0.6667     | 0.8962   | The model is much more effective for males. |
| **FPR**       | 0.0625     | 0.1000   | Females face fewer false positives than males. |
| **Precision** | 0.6667     | 0.9425   | Predictions are far more reliable for males. |
| **ROC AUC**   | 0.9531     | 0.9181   | Females benefit from higher ranking performance. |
| **TPR**       | 0.6667     | 0.8542   | Males are more likely to be correctly identified when positive. |

---

### 3. Interpretation
- **Females (unprivileged):**  
  - Show slightly higher accuracy and ROC AUC.  
  - However, they suffer from **lower F1-score (0.6667)** and **much lower TPR (66.7%)**, meaning many true cases are missed.  
  - Precision is also weaker (0.6667), so positive predictions for females are less trustworthy.  

- **Males (privileged):**  
  - Achieve stronger F1, precision, and sensitivity, indicating the model is **more effective overall** for this group.  
  - Face a somewhat higher false positive rate (10%), but this trade-off comes with much higher predictive reliability and sensitivity.

---

### **Summary**
The KNN model shows **clear bias in favor of males**.  
- Females benefit slightly in terms of ROC AUC and accuracy, but these gains are overshadowed by **much weaker recall, F1-score, and precision**, making the model less useful for them.  
- Males enjoy **more consistent and reliable predictions**, highlighting a systematic gender disparity in performance.

---

In [15]:
#group specific error analysis

from fairmlhealth import performance_metrics as pm

# Define group masks with clear names
female_mask = (protected_attr_knn == 0)  # unprivileged group (female)
male_mask   = (protected_attr_knn == 1)  # privileged group (male)

# Function to evaluate group-specific metrics
def evaluate_group_performance(group_name, mask):
    tpr = pm.true_positive_rate(y_test[mask], y_pred_knn[mask])
    fpr = pm.false_positive_rate(y_test[mask], y_pred_knn[mask])
    print(f"{group_name} Results:")
    print(f"  True Positive Rate (TPR): {tpr:.4f}")
    print(f"  False Positive Rate (FPR): {fpr:.4f}")
    print("-" * 40)

# Evaluate for each group
evaluate_group_performance("Female (Unprivileged)", female_mask)
evaluate_group_performance("Male (Privileged)", male_mask)


Female (Unprivileged) Results:
  True Positive Rate (TPR): 0.6667
  False Positive Rate (FPR): 0.0625
----------------------------------------
Male (Privileged) Results:
  True Positive Rate (TPR): 0.8542
  False Positive Rate (FPR): 0.1000
----------------------------------------


### Group-Specific Error Analysis – KNN Model

To examine fairness at the subgroup level, we compare the **True Positive Rate (TPR)** and **False Positive Rate (FPR)** for the unprivileged (female) and privileged (male) groups.

---

#### Results by Gender Group

| Group                  | TPR     | FPR     |
|------------------------|---------|---------|
| Unprivileged (Female)  | 0.6667  | 0.0625  |
| Privileged (Male)      | 0.8542  | 0.1000  |

---

#### Interpretation

- **True Positive Rate (TPR):**  
  - Females have a considerably lower TPR (66.67%) than males (85.42%).  
  - This means the model **misses more true positive cases** among females, suggesting lower sensitivity for this group.  

- **False Positive Rate (FPR):**  
  - Females experience a lower FPR (6.25%) compared to males (10.00%).  
  - This indicates that females are **less often incorrectly classified as positive**, whereas males face more false alarms.  

- **Trade-off observed:**  
  - The model is **more sensitive for males**, detecting more true cases, but at the cost of higher false positives.  
  - Conversely, females face **reduced detection of true cases** but benefit from fewer false positives.

---

#### Summary

The KNN model exhibits **unequal error distributions across genders**:  
- Females are disadvantaged in terms of **recall (TPR)**, meaning they are more likely to have their condition overlooked.  
- Males benefit from higher sensitivity but also face more false alarms.  

This imbalance contributes to fairness concerns, as the model **systematically favors the privileged group (males)** in identifying true cases.

---

### Decision Tree - DT

In [16]:
import pandas as pd

# Load KNN results
dt_df = pd.read_csv("HeartFailureData_50_50_AltTunedDT_predictions.csv")

print(dt_df.head())

   Sex  y_true    y_prob  y_pred
0    1       1  1.000000       1
1    1       1  0.000000       0
2    1       1  1.000000       1
3    1       1  0.272727       0
4    0       0  1.000000       1


In [17]:
import re

# Extract common columns
y_true_dt = dt_df["y_true"].values
y_prob_dt = dt_df["y_prob"].values
y_pred_dt = dt_df["y_pred"].values
gender_dt = dt_df["Sex"].values


# Use gender_knn as the protected attribute (0/1 as in your CSV)
protected_attr_dt = gender_dt


In [18]:
# Decision Tree Gender Bias Report
print("\n--- Decision Tree Gender Bias Report ---")

dt_bias = measure.summary(
    X=X_test,
    y_true=y_test,
    y_pred=y_pred_dt,
    y_prob=y_prob_dt,
    prtc_attr=protected_attr_dt,
    pred_type="classification",
    priv_grp=1,  # 1 = Male = Privileged
    sig_fig=4,
    skip_if=True,  
    skip_performance = True
)

print(dt_bias)


--- Decision Tree Gender Bias Report ---
                                                        Value
Metric         Measure                                       
Group Fairness AUC Difference                         -0.0635
               Balanced Accuracy Difference           -0.0646
               Balanced Accuracy Ratio                 0.9227
               Disparate Impact Ratio                  0.3891
               Equal Odds Difference                  -0.1042
               Equal Odds Ratio                        1.2500
               Positive Predictive Parity Difference  -0.4367
               Positive Predictive Parity Ratio        0.5338
               Statistical Parity Difference          -0.3306
Data Metrics   Prevalence of Privileged Class (%)     79.0000


In [19]:
#Flag metrics outside acceptable fairness bounds in current table 

styled_dt = MyFlagger().apply_flag(
    df=dt_bias,
    caption="Decision Tree Fairness (Gender)",
    boundaries=bounds,
    sig_fig=4,
    as_styler=True
)
styled_dt

## Gender Bias Detection Results – Decision Tree Model

This table summarizes group fairness metrics for the **Decision Tree (DT)** model, stratified by gender.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.

---

### 1. Group Fairness Metrics

- **AUC Difference (−0.0635):**  
  The ROC AUC is lower for females, indicating weaker ranking performance compared to males.

- **Balanced Accuracy Difference (−0.0646)** and **Ratio (0.9227):**  
  Females experience **notably worse balanced accuracy**, showing that the model is less effective in correctly identifying both positives and negatives for this group.

- **Disparate Impact Ratio (0.3891):**  
  Well below the fairness threshold (0.80–1.25), suggesting **substantially unequal selection rates**, with females selected much less frequently than males.

- **Equal Odds Difference (−0.1042)** and **Equal Odds Ratio (1.2500):**  
  Indicates a **moderate disparity in error rates** (TPR and FPR) across genders, with outcomes skewed toward males.

- **Positive Predictive Parity Difference (−0.4367)** and **Ratio (0.5338):**  
  Predictions are **far less reliable for females**, showing significantly reduced precision compared to males.

- **Statistical Parity Difference (−0.3306):**  
  Confirms that **females are selected at a much lower rate**, reinforcing evidence of systemic imbalance.

---

### 2. Interpretation

- The **Decision Tree model demonstrates clear fairness concerns**:  
  - Females suffer from **lower AUC and balanced accuracy**, meaning worse overall predictive performance.  
  - They face **reduced selection rates and weaker precision**, highlighting disadvantages in how positive predictions are distributed.  
  - While error rate disparities (Equal Odds) are not extreme, they consistently disadvantage females.  

---

### **Summary**

The Decision Tree model shows a **systematic bias against the unprivileged group (females)**.  
- Females face **reduced ranking performance, accuracy, and reliability of predictions**, alongside lower selection rates.  
- Males, as the privileged group, benefit from more favorable outcomes across multiple fairness dimensions.  

Overall, these results indicate that the Decision Tree introduces **significant gender bias** and would require **mitigation strategies** before deployment decision-making contexts.

---

In [20]:
print("FairMLHealth Stratified Bias Table - DT")
measure.bias(X_test, y_test, y_pred_dt, features=['Sex'], flag_oor=False)

FairMLHealth Stratified Bias Table - DT


,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,Sex,0,0.0646,1.0838,-0.025,0.80,0.4367,1.8734,0.3306,2.5702,0.1042,1.1563
1,Sex,1,-0.0646,0.9227,0.025,1.25,-0.4367,0.5338,-0.3306,0.3891,-0.1042,0.8649


## Stratified Bias Analysis – Decision Tree by Gender

This table presents **group-specific fairness metrics** for the Decision Tree (DT) model, stratified by gender.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.

---

### 1. Balanced Accuracy
- **Females (0):** Balanced Accuracy Difference = **+0.0646**, Ratio = **1.0838**  
- **Males (1):** Balanced Accuracy Difference = **−0.0646**, Ratio = **0.9227**  
- ➝ The model achieves **better balanced accuracy for females**, while males experience reduced performance.

---

### 2. False Positive Rate (FPR)
- **Females (0):** FPR Difference = **−0.0250**, Ratio = **0.80**  
- **Males (1):** FPR Difference = **+0.0250**, Ratio = **1.25**  
- ➝ Females face a **slightly lower false positive rate**, while males are more likely to be incorrectly flagged.

---

### 3. Positive Predictive Value (PPV / Precision)
- **Females (0):** PPV Difference = **+0.4367**, Ratio = **1.8734**  
- **Males (1):** PPV Difference = **−0.4367**, Ratio = **0.5338**  
- ➝ Predictions are **much more reliable for females**, while males show substantially lower precision.

---

### 4. Selection Rate
- **Females (0):** Selection Difference = **+0.3306**, Ratio = **2.5702**  
- **Males (1):** Selection Difference = **−0.3306**, Ratio = **0.3891**  
- ➝ Females are **selected far more frequently**, whereas males are under-selected.

---

### 5. True Positive Rate (TPR / Sensitivity)
- **Females (0):** TPR Difference = **+0.1042**, Ratio = **1.1563**  
- **Males (1):** TPR Difference = **−0.1042**, Ratio = **0.8649**  
- ➝ The model is **more sensitive for females**, correctly identifying a higher proportion of true positives compared to males.

---

### **Summary**
- The Decision Tree model introduces a **reverse bias**, with **females (unprivileged group)** benefiting from higher balanced accuracy, lower false positive rates, better precision, higher selection rates, and stronger sensitivity.  
- **Males (privileged group)** are consistently disadvantaged, facing more false positives, lower precision, and fewer positive selections.  

Overall, the DT model systematically **favors females over males**, highlighting fairness concerns that should be addressed to ensure more equitable performance across genders.

---

In [21]:
# Get the stratified performance table
perf_table_dt = measure.performance(
    X=gender_df,
    y_true=y_test,
    y_pred=y_pred_dt,
    y_prob=y_prob_dt
)

# Replace NaN with a dash
perf_table_dt = perf_table_dt.fillna("—")

# Display pretty table
display(perf_table_dt)

C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\fairmlhealth\__utils.py:57: UserWarning: Possible error in column(s) ['Sex']. DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,184.0,0.5543,0.4728,0.8207,0.8254,0.1098,—,0.8966,0.8724,0.7647
1,Sex,0,38.0,0.1579,0.2105,0.8421,0.5714,0.1250,—,0.5000,0.8151,0.6667
2,Sex,1,146.0,0.6575,0.5411,0.8151,0.8457,0.1000,—,0.9367,0.8786,0.7708


## Stratified Performance Analysis – Decision Tree by Gender

This table shows the **performance metrics** of the Decision Tree (DT) model stratified by gender.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.

---

### 1. Overall Performance (All Features)
- **Accuracy (0.8207)** and **F1-score (0.8254)** indicate moderate classification performance overall.  
- **ROC AUC (0.8724)** suggests the model has good discriminative ability.  
- **Precision (0.8966)** is relatively high, but subgroup analysis reveals substantial disparities.  
- **TPR (0.7647)** shows moderate sensitivity, meaning not all true cases are being detected.  

---

### 2. Subgroup Comparison

| Metric        | Female (0) | Male (1) | Interpretation |
|---------------|------------|----------|----------------|
| **Accuracy**  | 0.8421     | 0.8151   | Accuracy is slightly higher for females. |
| **F1-Score**  | 0.5714     | 0.8457   | The model performs much better for males in terms of balanced precision/recall. |
| **FPR**       | 0.1250     | 0.1000   | Females experience a slightly higher false positive rate. |
| **Precision** | 0.5000     | 0.9367   | Predictions for males are far more reliable; females’ predictions are weak. |
| **ROC AUC**   | 0.8151     | 0.8786   | Males benefit from stronger ranking performance. |
| **TPR**       | 0.6667     | 0.7708   | Sensitivity is higher for males, meaning they are more likely to be correctly identified. |

---

### 3. Interpretation
- **Females (unprivileged group):**  
  - Experience **weaker overall performance**, with much lower F1-score and precision.  
  - Their predictions are **less trustworthy** (precision only 0.5000), and they also face a slightly higher false positive rate.  
  - Lower TPR (66.7%) means more true female cases are missed compared to males.  

- **Males (privileged group):**  
  - Benefit from **substantially stronger performance**: higher F1, precision, ROC AUC, and TPR.  
  - Predictions for males are highly reliable, with strong sensitivity and ranking ability.  

---

### **Summary**
The Decision Tree model introduces a **systematic disadvantage for females**, who face:  
- **Lower predictive reliability (precision and F1)**,  
- **Higher false positives**, and  
- **Lower sensitivity (TPR)**.  

Males consistently receive more favorable outcomes, confirming that the DT model exhibits **gender bias in favor of males**.  

---

In [22]:
from fairmlhealth import performance_metrics as pm

# Define group masks with clear names
female_mask = (protected_attr_dt == 0)  # female = unprivileged group
male_mask   = (protected_attr_dt == 1)  # male = privileged group 

# Function to evaluate group-specific metrics
def evaluate_group_performance(group_name, mask):
    tpr = pm.true_positive_rate(y_test[mask], y_pred_dt[mask])
    fpr = pm.false_positive_rate(y_test[mask], y_pred_dt[mask])
    print(f"{group_name} Results:")
    print(f"  True Positive Rate (TPR): {tpr:.4f}")
    print(f"  False Positive Rate (FPR): {fpr:.4f}")
    print("-" * 40)

# Evaluate for each group
evaluate_group_performance("Female (Unprivileged)", female_mask)
evaluate_group_performance("Male (Privileged)", male_mask)

Female (Unprivileged) Results:
  True Positive Rate (TPR): 0.6667
  False Positive Rate (FPR): 0.1250
----------------------------------------
Male (Privileged) Results:
  True Positive Rate (TPR): 0.7708
  False Positive Rate (FPR): 0.1000
----------------------------------------


### Group-Specific Error Analysis – Decision Tree Model

To further examine subgroup fairness, we compare the **True Positive Rate (TPR)** and **False Positive Rate (FPR)** between the unprivileged (female) and privileged (male) groups.

---

#### Results by Gender Group

| Group                  | TPR     | FPR     |
|------------------------|---------|---------|
| Unprivileged (Female)  | 0.6667  | 0.1250  |
| Privileged (Male)      | 0.7708  | 0.1000  |

---

#### Interpretation

- The **TPR is higher for males (77.08%)** compared to females (66.67%).  
  - This means males are more likely to be correctly identified when they actually have CVD, while females face a greater risk of missed diagnoses.  
- The **FPR is slightly lower for males (10.00%)** than for females (12.50%).  
  - This indicates that females are somewhat more likely to be incorrectly flagged as having CVD.  
- Together, these disparities show that the model is **both more sensitive and more specific for males**, while females experience more errors in both directions.

---

#### Summary

The Decision Tree model shows a **systematic disadvantage for the unprivileged group (females)**:  
- They are **less likely to be correctly diagnosed (lower TPR)**,  
- And **more likely to be misclassified with false alarms (higher FPR)**.  

This reflects a fairness concern, as error distribution favors the privileged group (males), underscoring the need for **bias mitigation strategies**.

---

### Ensemble Model - Random Forest - RF

In [23]:
rf_df = pd.read_csv("HeartFailureData_50_50_RF_predictions.csv")
print(rf_df.head())

   Sex  y_true  y_prob  y_pred
0    1       1    0.96       1
1    1       1    0.19       0
2    1       1    0.97       1
3    1       1    0.31       0
4    0       0    0.37       0


In [24]:
# Extract common columns
y_true_rf = rf_df["y_true"].values
y_pred_rf = rf_df["y_pred"].values
y_prob_rf = rf_df["y_prob"].values
gender_rf = rf_df["Sex"].values


# Use gender_knn as the protected attribute (0/1 as in your CSV)
protected_attr_rf = gender_rf

In [25]:
# Random Forest Gender Bias Report
print("\n--- Random Forest Gender Bias Report ---")

rf_bias = measure.summary(
    X=X_test,
    y_true=y_test,
    y_pred=y_pred_rf,
    y_prob=y_prob_rf,
    prtc_attr=protected_attr_rf,
    pred_type="classification",
    priv_grp=1,  # 1 = Male = Privileged
    sig_fig=4,
    skip_if=True,
    skip_performance = True
)

print(rf_bias)


--- Random Forest Gender Bias Report ---
                                                        Value
Metric         Measure                                       
Group Fairness AUC Difference                          0.0663
               Balanced Accuracy Difference            0.0119
               Balanced Accuracy Ratio                 1.0144
               Disparate Impact Ratio                  0.4317
               Equal Odds Difference                  -0.0237
               Equal Odds Ratio                        0.8681
               Positive Predictive Parity Difference  -0.3989
               Positive Predictive Parity Ratio        0.5562
               Statistical Parity Difference          -0.3464
Data Metrics   Prevalence of Privileged Class (%)     79.0000


In [26]:
# Flagged fairness table for Random Forest
styled_rf = MyFlagger().apply_flag(
    df=rf_bias,
    caption="Random Forest Fairness (Gender)",
    boundaries=bounds,
    sig_fig=4,
    as_styler=True
)
styled_rf

## Gender Bias Detection Results – Random Forest Model

---

### 1. Group Fairness Metrics

- **AUC Difference (0.0663):** The ROC AUC differs slightly between genders, suggesting somewhat better ranking performance for one group.  
- **Balanced Accuracy Difference (0.0119)** and **Ratio (1.0144):** Very small disparity, indicating similar balanced accuracy across genders.  
- **Disparate Impact Ratio (0.4317):** Far below the fairness threshold of 0.80–1.25, showing **substantial inequality in selection rates**, with females disadvantaged.  
- **Equal Odds Difference (−0.0237)** and **Ratio (0.8681):** Small disparity in error rates (TPR and FPR), still favoring males slightly.  
- **Positive Predictive Parity Difference (−0.3989)** and **Ratio (0.5562):** Predictions are much less reliable for females, reflecting lower precision.  
- **Statistical Parity Difference (−0.3464):** Females are selected at a much lower rate than males, reinforcing evidence of systemic imbalance.

---

### 2. Interpretation

- The Random Forest model exhibits **clear fairness concerns**:  
  - **Selection rates and precision** heavily favor males (privileged group).  
  - While **balanced accuracy and AUC disparities are small**, suggesting broadly similar predictive ability,  
  - The **disparate impact and statistical parity values highlight structural disadvantages** for females.  
- Females experience both **reduced chances of being selected** and **lower predictive reliability**, which points to biased treatment despite overall model strength.

---

### **Summary**

The Random Forest model demonstrates **systematic gender bias**, with females (unprivileged group) disadvantaged in **selection rates and predictive parity**.  
Although differences in balanced accuracy and error rates appear small, the **large gaps in statistical and disparate impact measures** confirm that the model **favors males significantly**, warranting fairness mitigation before deployment.

---

In [27]:
print("FairMLHealth Stratified Bias Table - RF")
measure.bias(X_test, y_test, y_pred_rf, features=['Sex'], flag_oor=False)

FairMLHealth Stratified Bias Table - RF


,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,Sex,0,-0.0119,0.9858,0.0237,1.1520,0.3989,1.7978,0.3464,2.3164,0.0,1.0
1,Sex,1,0.0119,1.0144,-0.0237,0.8681,-0.3989,0.5562,-0.3464,0.4317,0.0,1.0


## Stratified Bias Analysis – Random Forest by Gender

This table presents **group-specific fairness metrics** for the Random Forest model, stratified by gender.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.

---

### 1. Balanced Accuracy
- **Females (0):** Balanced Accuracy Difference = **−0.0119**, Ratio = **0.9858**  
- **Males (1):** Balanced Accuracy Difference = **+0.0119**, Ratio = **1.0144**  
- ➝ Very small differences, suggesting **nearly equal balanced accuracy** across genders.

---

### 2. False Positive Rate (FPR)
- **Females (0):** FPR Difference = **+0.0237**, Ratio = **1.1520**  
- **Males (1):** FPR Difference = **−0.0237**, Ratio = **0.8681**  
- ➝ Females experience **slightly higher false positive rates**, meaning they are somewhat more often incorrectly flagged as positive.

---

### 3. Positive Predictive Value (PPV / Precision)
- **Females (0):** PPV Difference = **+0.3989**, Ratio = **1.7978**  
- **Males (1):** PPV Difference = **−0.3989**, Ratio = **0.5562**  
- ➝ Precision is **much higher for females**, suggesting predictions for women are more reliable, while males suffer from reduced predictive reliability.

---

### 4. Selection Rate
- **Females (0):** Selection Difference = **+0.3464**, Ratio = **2.3164**  
- **Males (1):** Selection Difference = **−0.3464**, Ratio = **0.4317**  
- ➝ Females are **selected more than twice as often** as males, indicating a strong bias in favor of women in this model.

---

### 5. True Positive Rate (TPR / Sensitivity)
- **Both groups:** TPR Difference = **0.0**, Ratio = **1.0**  
- ➝ Sensitivity is **equal across genders**, meaning both males and females have the same chance of being correctly identified when they truly have the condition.

---

### **Summary**
- The Random Forest model shows **balanced accuracy and sensitivity (TPR) across genders**, suggesting no disparity in detecting true cases.  
- However, females benefit from **much higher selection rates and precision**, while males are disadvantaged with lower predictive reliability.  
- Although the error rates are relatively close, the **large disparities in selection and precision** indicate that the model introduces a **systematic bias in favor of females**.

This reflects that while the RF model treats genders similarly in sensitivity, its **decision outcomes (selection rates and precision) are strongly skewed**, requiring fairness adjustments.

---

In [28]:
# Get the stratified performance table
perf_table_rf = measure.performance(
    X=gender_df,
    y_true=y_test,
    y_pred=y_pred_rf,
    y_prob=y_prob_rf
)

# Replace NaN with a dash
perf_table_rf = perf_table_rf.fillna("—")

# display pretty table
display(perf_table_rf)

C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\fairmlhealth\__utils.py:57: UserWarning: Possible error in column(s) ['Sex']. DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,184.0,0.5543,0.5380,0.8315,0.8458,0.1707,—,0.8586,0.9165,0.8333
1,Sex,0,38.0,0.1579,0.2632,0.8421,0.6250,0.1562,—,0.5000,0.9661,0.8333
2,Sex,1,146.0,0.6575,0.6096,0.8288,0.8649,0.1800,—,0.8989,0.8999,0.8333


## Stratified Performance Analysis – Random Forest by Gender  

This table shows the **stratified performance metrics** of the Random Forest (RF) model across gender groups.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.

---

### 1. Overall Performance (All Features)
- **Accuracy (0.8315)** and **F1-score (0.8458)** show good overall predictive performance.  
- **ROC AUC (0.9165)** indicates strong discriminatory ability.  
- **Precision (0.8586)** is high, suggesting reliable positive predictions.  
- **TPR (0.8333)** shows the model correctly identifies most true cases.  
- **Note**: PR AUC is not available (“—”), likely due to subgroup size limitations.

---

### 2. Subgroup Comparison

| Metric        | Female (0) | Male (1) | Interpretation |
|---------------|------------|----------|----------------|
| **Accuracy**  | 0.8421     | 0.8288   | Accuracy is slightly higher for females. |
| **F1-Score**  | 0.6250     | 0.8649   | Model performs much better for males in terms of F1. |
| **FPR**       | 0.1562     | 0.1800   | Females experience slightly fewer false positives than males. |
| **Precision** | 0.5000     | 0.8989   | Precision is much lower for females, meaning predictions for them are less reliable. |
| **ROC AUC**   | 0.9661     | 0.8999   | The model ranks females better than males. |
| **TPR**       | 0.8333     | 0.8333   | Sensitivity is equal across genders. |

---

### 3. Interpretation
- **Females (unprivileged):**
  - Benefit from slightly higher accuracy and much higher ROC AUC (0.9661).  
  - However, they face a **low F1-score (0.6250)** due to poor precision (0.5000), meaning the model produces many false alarms for women.  
  - False positive rate is slightly lower than for men.

- **Males (privileged):**
  - Receive **more balanced and reliable predictions**, with very high precision (0.8989) and strong F1-score (0.8649).  
  - Their ROC AUC is lower than for females, meaning the ranking ability is weaker.  
  - Slightly higher FPR indicates more false positives compared to females.

---

### **Summary**
The Random Forest model provides **equal sensitivity (TPR)** across genders but exhibits trade-offs:  
- **Females** benefit from higher ranking quality (ROC AUC) and fewer false positives but suffer from **very low precision and weak F1 performance**.  
- **Males** receive more consistent performance with strong precision and F1 but are ranked less effectively.  

This indicates that while the RF model does not disadvantage either group in sensitivity, it shows **imbalances in prediction quality**: females face unreliable predictions, while males face slightly higher misclassification rates.

---

In [29]:
from fairmlhealth import performance_metrics as pm

# Define group masks with clear names
female_mask = (protected_attr_rf == 0)  # female = unprivileged group
male_mask   = (protected_attr_rf == 1)  # male = privileged group 

# Function to evaluate group-specific metrics
def evaluate_group_performance(group_name, mask):
    tpr = pm.true_positive_rate(y_test[mask], y_pred_rf[mask])
    fpr = pm.false_positive_rate(y_test[mask], y_pred_rf[mask])
    print(f"{group_name} Results:")
    print(f"  True Positive Rate (TPR): {tpr:.4f}")
    print(f"  False Positive Rate (FPR): {fpr:.4f}")
    print("-" * 40)

# Evaluate for each group
evaluate_group_performance("Female (Unprivileged)", female_mask)
evaluate_group_performance("Male (Privileged)", male_mask)

Female (Unprivileged) Results:
  True Positive Rate (TPR): 0.8333
  False Positive Rate (FPR): 0.1562
----------------------------------------
Male (Privileged) Results:
  True Positive Rate (TPR): 0.8333
  False Positive Rate (FPR): 0.1800
----------------------------------------


### Group-Specific Error Analysis – Random Forest (RF) Model

To assess fairness at the subgroup level, we compared the **True Positive Rate (TPR)** and **False Positive Rate (FPR)** for females (unprivileged) and males (privileged).

---

#### Results by Gender Group

| Group                  | TPR     | FPR     |
|------------------------|---------|---------|
| Unprivileged (Female)  | 0.8333  | 0.1562  |
| Privileged (Male)      | 0.8333  | 0.1800  |

---

#### Interpretation

- **True Positive Rate (TPR):** Both groups achieve the **same sensitivity (83.33%)**, meaning the model is equally effective at detecting true positive cases for females and males.  
- **False Positive Rate (FPR):** The rate of false alarms is slightly **lower for females (15.62%)** compared to males (18.00%). This means men are marginally more likely to be incorrectly classified as positive.  
- The error distribution here is relatively balanced, with **minimal disparity in TPR** and only a **small difference in FPR**.  

---

#### Summary

The Random Forest model demonstrates **fairly equitable performance across genders** in terms of sensitivity and specificity. While males experience a slightly higher false positive rate, the differences are minor, indicating that this model shows **low levels of gender bias** based on TPR and FPR.


---

### Deep Learning Model - Feed Forward Network (MLP)

In [30]:
mlp_df = pd.read_csv("HeartFailureData_50_50_RecallFirstTunedMLP_predictions.csv")
print(mlp_df.head())

   Sex  y_true    y_prob  y_pred
0    1       1  0.976401       1
1    1       1  0.011376       0
2    1       1  0.839763       1
3    1       1  0.144366       0
4    0       0  0.204195       0


In [31]:
# Extract common columns 
y_true_mlp = mlp_df["y_true"].values 
y_prob_mlp = mlp_df["y_prob"].values
y_pred_mlp = mlp_df["y_pred"].values
gender_mlp = mlp_df["Sex"].values 

# Use gender_mlp as the protected attribute
protected_attr_mlp = gender_mlp 

In [32]:
#Run fairmlhealth bias detection for MLP 

mlp_bias = measure.summary(
    X=X_test,
    y_true=y_test,
    y_pred=y_pred_mlp,
    y_prob=y_prob_mlp,
    prtc_attr=protected_attr_mlp,
    pred_type="classification",
    priv_grp=1,
    sig_fig=4,
    skip_if=True,
    skip_performance = True
)

print(mlp_bias)

                                                        Value
Metric         Measure                                       
Group Fairness AUC Difference                          0.0175
               Balanced Accuracy Difference           -0.0650
               Balanced Accuracy Ratio                 0.9237
               Disparate Impact Ratio                  0.3164
               Equal Odds Difference                  -0.1562
               Equal Odds Ratio                        0.7812
               Positive Predictive Parity Difference  -0.3580
               Positive Predictive Parity Ratio        0.6148
               Statistical Parity Difference          -0.3980
Data Metrics   Prevalence of Privileged Class (%)     79.0000


In [33]:
# Flagged fairness table for MLP
styled_mlp = MyFlagger().apply_flag(
    df=mlp_bias,
    caption="MLP Fairness (Gender)",
    boundaries=bounds,
    sig_fig=4,
    as_styler=True
)
styled_mlp

## Gender Fairness Analysis – MLP Model

This table reports the fairness metrics of the **MLP model** stratified by gender.  
Here, males represent the **privileged group** and females the **unprivileged group**.

---

### 1. Performance Disparities
- **AUC Difference (0.0175):** Very small difference, indicating that the model ranks predictions similarly across genders.  
- **Balanced Accuracy Difference (−0.0650) & Ratio (0.9237):** Balanced accuracy is lower for females, meaning the model detects outcomes less effectively for them.

---

### 2. Group Fairness
- **Disparate Impact Ratio (0.3164):** Well below the acceptable fairness threshold (0.80–1.25). This indicates **females are selected at a much lower rate** than males.  
- **Equal Odds Difference (−0.1562) & Ratio (0.7812):** Error rates (TPR and FPR) differ across genders, with males performing more favorably.  
- **Positive Predictive Parity Difference (−0.3580) & Ratio (0.6148):** Precision is significantly lower for females, suggesting their positive predictions are less reliable.  
- **Statistical Parity Difference (−0.3980):** Strong evidence of under-selection for females, as they are less likely to receive positive predictions.

---

### 3. Interpretation
- The MLP model shows **systematic disadvantages for females**:  
  - Lower **balanced accuracy**, meaning the model struggles more to correctly identify true outcomes for women.  
  - Strong disparities in **selection rates, precision, and error balance**, indicating both reliability and fairness gaps.  
  - Metrics like **Disparate Impact Ratio (0.3164)** and **Statistical Parity Difference (−0.3980)** confirm structural bias against females.

---

### **Summary**
The MLP model demonstrates **substantial gender bias**, with females receiving fewer positive predictions, less reliable results, and reduced accuracy.  


---

In [34]:
print("FairMLHealth Stratified Bias Table - MLP")
measure.bias(X_test, y_test, y_pred_mlp, features=['Sex'], flag_oor=False)

FairMLHealth Stratified Bias Table - MLP


,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,Sex,0,0.065,1.0826,0.0262,1.2800,0.358,1.6265,0.398,3.1605,0.1562,1.2344
1,Sex,1,-0.065,0.9237,-0.0262,0.7812,-0.358,0.6148,-0.398,0.3164,-0.1562,0.8101


## Stratified Bias Analysis – MLP by Gender

This table presents **group-specific fairness metrics** for the MLP model, stratified by gender.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.

---

### 1. Balanced Accuracy
- **Females (0):** Difference = **+0.065**, Ratio = **1.0826**  
- **Males (1):** Difference = **−0.065**, Ratio = **0.9237**  
- ➝ The model achieves **better balanced accuracy for females**, while males face a relative disadvantage.

---

### 2. False Positive Rate (FPR)
- **Females (0):** FPR Difference = **+0.0262**, Ratio = **1.2800**  
- **Males (1):** FPR Difference = **−0.0262**, Ratio = **0.7812**  
- ➝ Females have a **slightly higher false positive rate**, meaning they are more often misclassified as positive compared to males.

---

### 3. Positive Predictive Value (PPV / Precision)
- **Females (0):** PPV Difference = **+0.358**, Ratio = **1.6265**  
- **Males (1):** PPV Difference = **−0.358**, Ratio = **0.6148**  
- ➝ Predictions are **much more precise for females**, while males experience significantly less reliable positive predictions.

---

### 4. Selection Rate
- **Females (0):** Selection Difference = **+0.398**, Ratio = **3.1605**  
- **Males (1):** Selection Difference = **−0.398**, Ratio = **0.3164**  
- ➝ Females are **selected much more frequently**, whereas males are under-selected relative to their target distribution.

---

### 5. True Positive Rate (TPR / Sensitivity)
- **Females (0):** TPR Difference = **+0.1562**, Ratio = **1.2344**  
- **Males (1):** TPR Difference = **−0.1562**, Ratio = **0.8101**  
- ➝ The model is **considerably more sensitive for females**, detecting more true positives compared to males.

---

### **Summary**
The MLP model shows a **clear bias in favor of females (unprivileged group)**:  
- They benefit from **higher balanced accuracy, higher precision, stronger sensitivity, and much higher selection rates**.  
- However, this comes with a **slightly elevated false positive rate**.  
- Conversely, males (privileged group) face **lower selection, weaker predictive reliability, and reduced sensitivity**, making them systematically disadvantaged.

In essence, the MLP introduces a **reverse bias**, strongly favoring females over males in classification outcomes.

---

In [35]:
# Get the stratified performance table
perf_table_mlp = measure.performance(
    X=gender_df,
    y_true=y_test,
    y_pred=y_pred_mlp,
    y_prob=y_prob_mlp
)

# Replace NaN with a dash
perf_table_mlp = perf_table_mlp.fillna("—")

# display pretty table
display(perf_table_mlp)

C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\fairmlhealth\__utils.py:57: UserWarning: Possible error in column(s) ['Sex']. DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,184.0,0.5543,0.5000,0.8478,0.8557,0.1098,—,0.9022,0.9139,0.8137
1,Sex,0,38.0,0.1579,0.1842,0.8684,0.6154,0.0938,—,0.5714,0.9167,0.6667
2,Sex,1,146.0,0.6575,0.5822,0.8425,0.8729,0.1200,—,0.9294,0.8992,0.8229


## Stratified Performance Analysis – MLP by Gender

This table shows the **stratified performance metrics** of the MLP model across gender groups.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.

---

### 1. Overall Performance (All Features)
- **Accuracy (0.848)** and **F1-score (0.856)** indicate strong overall classification ability.  
- **ROC AUC (0.914)** shows very good discriminatory power.  
- **Precision (0.902)** is high, suggesting predictions are generally reliable.  
- **TPR (0.814)** reflects good sensitivity, though subgroup comparisons highlight disparities.  
- **Note**: PR AUC is not available (“—”).

---

### 2. Subgroup Comparison

| Metric        | Female (0) | Male (1) | Interpretation |
|---------------|------------|----------|----------------|
| **Accuracy**  | 0.8684     | 0.8425   | Accuracy is slightly higher for females. |
| **F1-Score**  | 0.6154     | 0.8729   | Model performs substantially better for males. |
| **FPR**       | 0.0938     | 0.1200   | Females face fewer false positives than males. |
| **Precision** | 0.5714     | 0.9294   | Predictions for males are much more reliable. |
| **ROC AUC**   | 0.9167     | 0.8992   | Females benefit from slightly higher ranking performance. |
| **TPR**       | 0.6667     | 0.8229   | Females are more likely to be missed (lower sensitivity). |

---

### 3. Interpretation
- **Females (unprivileged)**:  
  - Show **higher accuracy and ROC AUC**, but suffer from **low F1-score and precision**.  
  - Their **TPR (66.7%)** indicates many missed true cases, making the model less sensitive for them.  
  - Despite fewer false positives (lower FPR), their positive predictions are less reliable.  

- **Males (privileged)**:  
  - Enjoy **much stronger F1-score, higher precision, and greater sensitivity (82.3% TPR)**.  
  - They face more false positives than females, but overall predictions are much more trustworthy.  

---

### **Summary**
The MLP model reveals a **systematic disadvantage for females**:  
- While they achieve comparable accuracy and even slightly better ROC AUC, their **precision, recall, and F1-score are significantly worse**.  
- This means that **females are both more likely to be missed and less likely to receive reliable positive predictions**.  
- Males, in contrast, consistently benefit from stronger predictive reliability and higher sensitivity.  

This pattern indicates a fairness concern, where the model systematically favors the privileged group (males) in outcome quality.

---

In [36]:
from fairmlhealth import performance_metrics as pm

# Define group masks with clear names
female_mask = (protected_attr_mlp == 0)  # female = unprivileged group
male_mask   = (protected_attr_mlp == 1)  # male = privileged group 

# Function to evaluate group-specific metrics
def evaluate_group_performance(group_name, mask):
    tpr = pm.true_positive_rate(y_test[mask], y_pred_mlp[mask])
    fpr = pm.false_positive_rate(y_test[mask], y_pred_mlp[mask])
    print(f"{group_name} Results:")
    print(f"  True Positive Rate (TPR): {tpr:.4f}")
    print(f"  False Positive Rate (FPR): {fpr:.4f}")
    print("-" * 40)

# Evaluate for each group
evaluate_group_performance("Female (Unprivileged)", female_mask)
evaluate_group_performance("Male (Privileged)", male_mask)

Female (Unprivileged) Results:
  True Positive Rate (TPR): 0.6667
  False Positive Rate (FPR): 0.0938
----------------------------------------
Male (Privileged) Results:
  True Positive Rate (TPR): 0.8229
  False Positive Rate (FPR): 0.1200
----------------------------------------


### Group-Specific Error Analysis – MLP Model

To further assess fairness at the subgroup level, we compare the **True Positive Rate (TPR)** and **False Positive Rate (FPR)** for the unprivileged (female) and privileged (male) groups.

---

#### Results by Gender Group

| Group                  | TPR     | FPR     |
|------------------------|---------|---------|
| Unprivileged (Female)  | 0.6667  | 0.0938  |
| Privileged (Male)      | 0.8229  | 0.1200  |

---

#### Interpretation

- **True Positive Rate (TPR):**  
  - Males achieve a higher TPR (**82.29%**) compared to females (**66.67%**).  
  - This means the model is **more sensitive for males**, correctly identifying more true positive cases in this group.  
  - Females face a higher risk of missed diagnoses.  

- **False Positive Rate (FPR):**  
  - Females have a slightly lower FPR (**9.38%**) than males (**12.00%**).  
  - This indicates that females are **less likely to be incorrectly flagged** as positive when they are actually negative.  

- **Overall trade-off:**  
  - For males, the model sacrifices a **higher false positive rate** in exchange for a **much better TPR**.  
  - For females, the model has **fewer false alarms** but also **misses more true cases**, raising concerns for fairness in high-stakes domains like healthcare.

---

#### **Summary**
The MLP model demonstrates a **performance imbalance**:  
- **Males (privileged group)** benefit from **higher sensitivity**, ensuring more of their true cases are detected.  
- **Females (unprivileged group)** experience **lower sensitivity**, meaning more missed cases, although they face fewer false positives.  

This asymmetry highlights a **fairness concern**, as the model may systematically disadvantage females by under-diagnosing their true conditions.

---